In [1]:
import requests
import pandas as pd
import requests
import json
from datetime import datetime
import pytz

In [2]:
# Azure App credentials
client_id = "183f41fa-e4bc-4ed0-b384-fc87fc1ca4ab"
client_secret = "0n-8Q~TaxuV81dY5QwJdsNksuvRKyQEwOgBRGb8c"
tenant_id = "5eafb13a-8bcd-462a-9e16-58810b6f2460"
scope = "https://graph.microsoft.com/.default"

In [3]:
# Token endpoint
token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Request access token
token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': scope
}

token_response = requests.post(token_url, data=token_data)
access_token = token_response.json().get("access_token")

if not access_token:
    print("Failed to get access token.")
    print(token_response.json())
    exit()

In [4]:
# URL for GET request to view stored user data
url = 'https://userprofile-ezcl.onrender.com/api/view_data'

# Send GET request
response = requests.get(url)
response = json.loads(response.text)

# Original response
res = response[-1]

# Parse the timestamp
utc_time = datetime.strptime(res['timestamp'][:26], "%Y-%m-%dT%H:%M:%S.%f")
utc_time = utc_time.replace(tzinfo=pytz.UTC)

# Convert to IST
ist = pytz.timezone('Asia/Kolkata')
ist_time = utc_time.astimezone(ist)

latest_user = {
    "user_email": res['user_email'],
    "timestamp": ist_time.strftime("%Y-%m-%d %H:%M:%S")
}

In [5]:
# Microsoft Graph API endpoint (example: list users)
graph_endpoint = f"https://graph.microsoft.com/v1.0/users/{latest_user['user_email']}"

# Set headers with access token
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Make the API call
response = requests.get(graph_endpoint, headers=headers)

# Parse response
if response.status_code == 200:
    data = response.json().get("value", [])
    df = pd.DataFrame(data)
    display(response.json())
else:
    print("Failed to fetch data:", response.status_code)
    print(response.text)

{'@odata.context': 'https://graph.microsoft.com/v1.0/$metadata#users/$entity',
 'businessPhones': ['9798733872'],
 'displayName': 'Ankit Kumar',
 'givenName': 'Ankit',
 'jobTitle': 'Data Scientist',
 'mail': 'ankitkumar875740@gmail.com',
 'mobilePhone': '9798733872',
 'officeLocation': None,
 'preferredLanguage': 'en-US',
 'surname': 'Kumar',
 'userPrincipalName': 'AnkitKumar@ankit8544.onmicrosoft.com',
 'id': '294dba8e-a69e-408d-a3ce-50fbfe042615'}

'294dba8e-a69e-408d-a3ce-50fbfe042615'

In [9]:
import requests

url = f"https://graph.microsoft.com/v1.0/users/{response.json()['id']}/photo/$value"
headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open("profile_image.jpg", "wb") as f:
        f.write(response.content)
